# Acrobot - v1

Acribit es un pendulo con dos enlaces donde solo la segunda articulación es accionada inicialmente, ambos enlaces apuntan hacia abajo. El objetivo es balancear la punta final a una altura de al menos el largo del enlace superior sobre la base. Ambos enlaces pueden balancearse libremente y pueden pasarse entre ellos, i.e., no chocan cuando tienen el mismo angulo.

#### Librerias

* Gym de OpenAI para usar su ambiente del Acrobot, el cual ya viene hecho con todo lo necesario excepto el algoritmo de aprendizaje
* Numpy porque pues que es python sin numpy

In [28]:
import gym
import numpy as np

Cargar el ambiente

In [31]:
env = gym.make('Acrobot-v1');
env.reset()

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


array([0.99884679, 0.04801125, 0.99997516, 0.00704784, 0.07396199,
       0.08673978])

#### Escoger acción

In [12]:
ACTIONS = [0,1,2]

# probability for exploration
EPSILON = 0.1

# step size
ALPHA = 0.5

# gamma for Q-Learning and Expected Sarsa
GAMMA = 1

normalize_const = np.linalg.norm([0, np.pi*2])

def normalize(x):
    return x / normalize_const

def cut_digits(theta1, theta2):
    if theta1 < 0:
        theta1 = int(np.ceil(theta1*10000))+9999
    else:
        theta1 = int(np.floor(theta1*10000))+10000
    
    if theta2 < 0:
        theta2 = int(np.ceil(theta2*1000))+9999
    else:
        theta2 = int(np.floor(theta2*1000))+10000
        
    return theta1,theta2

# choose an action based on epsilon greedy algorithm
def choose_action(state, q_value):
    if np.random.binomial(1, EPSILON) == 1:
        return np.random.choice(ACTIONS)
    else:
        values_ = q_value[state[0], state[1], :]
        return np.random.choice([action_ for action_, value_ in enumerate(values_) if value_ == np.max(values_)])

#### Q-Learning

In [16]:
# an episode with Q-Learning
# @q_value: values for state action pair, will be updated
# @step_size: step size for updating
# @return: total rewards within this episode
def q_learning(q_value, step_size=ALPHA):
    state = env.reset()
    rewards = 0.0
    terminal = env.env._terminal()
    # Get the thetas from the state
    state = normalize(np.array([state[4], state[5]]))
    state = cut_digits(state[0],state[1])
    while not terminal:
        # Get the thetas from the state
        #p_state = cut_digits(state[4],state[5])
        
        action = choose_action(state, q_value)
        next_state, reward, terminal, _= env.step(action)
        rewards += reward
        next_state = normalize(np.array([next_state[4], next_state[5]]))
        next_state = cut_digits(next_state[0],next_state[1])
        # Q-Learning update
        q_value[state[0], state[1], action] += step_size * (
                reward + GAMMA * np.max(q_value[next_state[0], next_state[1], :]) -
                q_value[state[0], state[1], action])
        state = next_state
    return rewards

In [ ]:
def print_optimal_policy(q_value):
    env = gym.make('Acrobot-v1')
    state = env.reset()
    state = [state[4], state[5]]
    while !env.env._terminal():
        env.render()
        state = normalize(np.array([state[0], state[1]]))
        state = cut_digits(state[0],state[1])
        

In [26]:
episodes = 200

# perform 40 independent runs
runs = 50

rewards_q_learning = np.zeros(episodes)
for r in range(runs):
    q_q_learning = np.zeros((19999,19999,3))
    for i in range(0, episodes):
        # cut off the value by -100 to draw the figure more elegantly
        # rewards_sarsa[i] += max(sarsa(q_sarsa), -100)
        # rewards_q_learning[i] += max(q_learning(q_q_learning), -100)
        rewards_q_learning[i] += q_learning(q_q_learning)

# averaging over independt runs
rewards_q_learning /= runs


In [27]:
rewards_q_learning

array([-500.  , -496.72, -497.82, -498.52, -500.  , -500.  , -500.  ,
       -500.  , -499.48, -500.  , -500.  , -499.28, -500.  , -500.  ,
       -500.  , -500.  , -500.  , -496.08, -498.78, -500.  , -493.9 ,
       -500.  , -500.  , -500.  , -500.  , -500.  , -500.  , -500.  ,
       -500.  , -500.  , -500.  , -498.9 , -498.88, -500.  , -495.58,
       -500.  , -499.74, -498.96, -497.48, -499.42, -498.78, -497.58,
       -500.  , -500.  , -500.  , -499.92, -497.36, -500.  , -495.72,
       -499.48, -495.84, -498.78, -500.  , -500.  , -500.  , -500.  ,
       -500.  , -498.12, -500.  , -500.  , -499.8 , -498.76, -500.  ,
       -500.  , -499.54, -499.2 , -497.88, -500.  , -496.1 , -499.2 ,
       -494.98, -500.  , -500.  , -499.68, -495.92, -496.4 , -500.  ,
       -500.  , -500.  , -500.  , -500.  , -496.84, -500.  , -500.  ,
       -498.12, -500.  , -500.  , -497.96, -494.56, -496.08, -500.  ,
       -495.48, -500.  , -500.  , -500.  , -497.68, -496.84, -497.26,
       -497.54, -497

In [29]:
print('Q-Learning Optimal Policy:')
print_optimal_policy(q_q_learning)

ReraisedException: Error occured while running `from pyglet.gl import *`
The original exception was:

ImportError: Library "c" not found.

HINT: make sure you have OpenGL install. On Ubuntu, you can run 'apt-get install python-opengl'. If you're running on a server, you may need a virtual frame buffer; something like this should work: 'xvfb-run -s "-screen 0 1400x900x24" python <your_script.py>'